In [ ]:
# Import all necessary libraries and modules
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from urllib.parse import urlparse, parse_qs
from dotenv import load_dotenv

In [ ]:
# Load environment variables from a .env file
load_dotenv()

## **Step 1A : Indexing ( Document Ingestion )**

In [ ]:
# Give the video url here:
video_url = "https://www.youtube.com/watch?v=Gfr50f6ZBvo"

In [ ]:
# Function to extract video ID from URL
def get_video_id(url: str) -> str:
    parsed_url = urlparse(url)
    if parsed_url.hostname in ["www.youtube.com", "youtube.com"]:
        return parse_qs(parsed_url.query)["v"][0]   # normal youtube link
    elif parsed_url.hostname == "youtu.be":
        return parsed_url.path[1:]                  # short link
    else:
        raise ValueError("Invalid YouTube URL")

# Call the function with the provided URL
video_id = get_video_id(video_url)
video_id


In [ ]:
# Fetch and print the transcript
try:
    # Load the transcript using the YouTubeTranscriptApi
    ytt_api = YouTubeTranscriptApi()
    response = ytt_api.fetch(video_id,languages=["en"])

    # Combine all text snippets into a single transcript
    transcript = " ".join([chunk.text for chunk in response.snippets])
    print(transcript)
except:
    print("No caption available for this video.")

## **Step 1B : Indexing ( Text Splitting )**

In [ ]:
# Initialize the text splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

# Split the transcript into smaller chunks
chunks = splitter.create_documents([transcript])

In [ ]:
# Print the number of chunks created
print(len(chunks))

## **Step 1C & 1D : Indexing ( Embedding Generation and Storing in Vector Store )**

In [ ]:
# Initialize the embedding model
embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

# Create a FAISS vector store from the chunks and embeddings
vectorstore = FAISS.from_documents(chunks, embedding)

In [ ]:
# Get the IDs of the documents in the vector store
vectorstore.index_to_docstore_id

## **Step 2 : Retriever**

In [ ]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 4}
)

In [ ]:
# Invoke the retriever with a sample query
retriever.invoke("What is DeepMind")

## **Step 3 : Augmentation**

In [ ]:
# Create a PromptTemplate
template = PromptTemplate(
    template=
    """
    You are a helpful assistant.
    Answer Only from the provided transcript context.
    If the context is insufficient, just say you don't know.
    Context: {context}
    Question: {question}
    """,
    input_variables=["context", "question"]
)

In [ ]:
# Define the question to be asked
question = "is the topic of aliens discussed in the video? if yes, then what was discussed?"

In [ ]:
# Retrieve relevant documents based on the question
retrieved_docs = retriever.invoke(question)

In [ ]:
# Combine the content of the retrieved documents into a single context string
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [ ]:
# Create the final Prompt
prompt = template.invoke(
    {
        "context": context_text,
        "question": question
    }
)

In [ ]:
prompt

## **Step 4 : Generation**

In [ ]:
# Initialize the chat model
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [ ]:
# Initialize a StrOutputParser
parser = StrOutputParser()

In [ ]:
# Generate a response using the model and the prompt
result = model.invoke(prompt)

In [ ]:
# Print the generated response
result.content